<a href="https://colab.research.google.com/github/ammobam/Display_SensorData/blob/main/210731_%EB%8D%B0%EC%9D%B4%ED%84%B0_%ED%83%90%EC%83%89(4)%EC%BB%AC%EB%9F%BC_%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **디스플레이 생산라인 센서 데이터 분석

## 주제 : 불량품에 영향을 주는 요인 찾기
- 결과 : "ㅇㅇ센서에서 ㅁㅁ면 불량품"
1. 주요 요인 피처 5개 추출
2. 불량품/양품 구분하는 모델도 찾아야 함


- 머신러닝 이용
---

## 컬럼이름 분류하기
- 데이터는 센서 데이터로 약  800여개의 컬럼이 있음
- 이름이 비슷한 컬럼이 다수 있으며 생산라인이 Left, Right으로 나뉘어 동일한 공정을 수행중임
- 컬럼이름을 기준으로 컬럼을 분류하고자 함
    - LEFT
    - RIGHT
    - 공통
- 컬럼을 분류하면 피처 수를 줄여 데이터탐색이 가능함
    - 이름이 유사한 컬럼간 종속성을 확인
    - LEFT, RIGHT 각 공정에 따라 레이블에 미치는 영향 확인 


In [10]:
# 데이터 불러오기
import pandas as pd
df_origin = pd.read_csv('/content/drive/MyDrive/나무플래닛/1. 데이터_디스플레이/factory_glass_2016.zip', encoding='cp949')

In [ ]:
# 컬럼이름 리스트 저장하기
col = df_origin.columns.tolist()
#col

In [12]:
# 1. 앞 3글자가 동일한 컬럼을 분류하고 확인
col_name_3=[]
for name in col:
    name = str(name)[:3]
    col_name_3.append(name)
# 앞글자 3자리가 같은 컬럼의 집단 개수
print("", len(set(col_name_3)))  # 61개

# 2. 컬럼명에 LEFT, RIGHT가 있으면 분류
# 3. 컬럼명에 _L, _R이 있으면 분류
    # _L, _R 포함한 이름들 먼저 확인해보기

 61
